In [ ]:
#This is Simon's code that we used to reach our results, conclusions and next steps for our Presentation

import os
import requests
import json
import pandas as pd
from dotenv import load_dotenv
import alpaca_trade_api as tradeapi


import numpy as np
%matplotlib inline
import hvplot.pandas
from decouple import config
from pathlib import Path
import hvplot.pandas
import matplotlib.pyplot as plt
from sklearn import svm
from sklearn.preprocessing import StandardScaler
from pandas.tseries.offsets import DateOffset
from sklearn.metrics import classification_report
from finta import TA

import pandas_datareader.data as web
import pandas as pd
import datetime as dt
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import style
#from MCForecastTools import mc_simulation

style.use('ggplot')

from mc_simulation import AverageModel as am
from mc_simulation import VolatilityModel as vm
from mc_simulation import MonteCarloSimulation as mcs

#API_USERNAME = config('USER')
#API_KEY = config('KEY')

In [ ]:
load_dotenv()


In [ ]:
#Read the file in dataframe
file_path = './Resources/aapl_stock_price_Jan_01_2021_to_April_7_2022.csv'
AAPL_df = pd.read_csv(Path(file_path),index_col='Date',parse_dates=True,infer__datetime_format=True)

# Review the DataFrame
AAPL_df['actual_returns'] = AAPL_df['close'].pct_change()
AAPL_df = AAPL_df.dropna()

AAPL_df.head()



In [ ]:
# Review the DataFrame
AAPL_df.head(10)
AAPL_df

#AAPL_df=pd.DataFrame(close, index=pd.DatetimeIndex)
AAPL_df["close"].plot()
AAPL_df

In [ ]:
import hvplot.pandas
AAPL_df.hvplot.line()

In [ ]:
AAPL_df["trade_type"]=np.nan

In [ ]:
previous_price=0

In [ ]:
for index,row in AAPL_df.iterrows():
    if previous_price==0:
        AAPL_df.loc[index,"trade_type"]="buy"
        
    elif row["close"] < previous_price:
            AAPL_df.loc[index,"trade_type"]="buy"
    

    elif row["close"] > (1.005*previous_price):
            AAPL_df.loc[index,"trade_type"]="sell"
    else:
         AAPL_df.loc[index, "trade_type"] = "hold"
            
previous_price=row["close"]

if index == AAPL_df.index[-1]:
    AAPL_df.loc[index, "trade_type"] = "sell"

AAPL_df.head(15)
            
        

In [ ]:
AAPL_df["cost/proceeds"]=np.nan
share_size = 1
accumulated_shares = 0


In [ ]:
for index, row in AAPL_df.iterrows():

    # buy if the previous_price is 0, in other words, buy on the first day
    if previous_price == 0:
        AAPL_df.loc[index, "trade_type"] = "buy"

        # calculate the cost of the trade by multiplying the current day's price
        # by the share_size, or number of shares purchased
        AAPL_df.loc[index, "cost/proceeds"] = -(row["close"] * share_size)

        # add the number of shares purchased to the accumulated shares
        accumulated_shares += share_size

    # buy if the current day's price is less than the previous day's price
    elif row["close"] < previous_price:
        AAPL_df.loc[index, "trade_type"] = "buy"

        # calculate the cost of the trade by multiplying the current day's price
        # by the share_size, or number of shares purchased
        AAPL_df.loc[index, "cost/proceeds"] = -(row["close"] * share_size)

        # add the number of shares purchased to the accumulated shares
        accumulated_shares += share_size
    elif row["close"] > (1.005*previous_price):
        AAPL_df.loc[index, "trade_type"] = "sell"

In [ ]:
if index == AAPL_df.index[-1]:
    AAPL_df.loc[index, "trade_type"] = "sell"

    # calculate the proceeds by multiplying the last day's price by the accumulated shares
   # amd_df.loc[index, "cost/proceeds"] = row["close"] * accumulated_shares

In [ ]:
#Initialize trade_type column to track buys and sells
AAPL_df["trade_type"] = np.nan

# Initialize a cost/proceeds column for recording trade metrics
AAPL_df["cost/proceeds"] = np.nan
AAPL_df["accumulated_shares"] = np.nan
# Initialize share size and accumulated shares
share_size = 1
accumulated_shares = 0

# Initialize variable to hold previous price
previous_price = 0

# Loop through the Pandas DataFrame and initiate a trade at each iteration
for index, row in AAPL_df.iterrows():

    # buy if the previous_price is 0, in other words, buy on the first day
    if previous_price == 0:
        AAPL_df.loc[index, "trade_type"] = "buy"

        # calculate the cost of the trade by multiplying the current day's price
        # by the share_size, or number of shares purchased
        AAPL_df.loc[index, "cost/proceeds"] = -(row["close"] * share_size)

        # add the number of shares purchased to the accumulated shares
        accumulated_shares += share_size
        AAPL_df.loc[index, "accumulated_shares"] = accumulated_shares
    # buy if the current day's price is less than the previous day's price
    elif row["close"] < 0.98*previous_price:
        AAPL_df.loc[index, "trade_type"] = "buy"

        # calculate the cost of the trade by multiplying the current day's price
        # by the share_size, or number of shares purchased
        AAPL_df.loc[index, "cost/proceeds"] = -(row["close"] * share_size)

        # add the number of shares purchased to the accumulated shares
        accumulated_shares += share_size
        AAPL_df.loc[index, "accumulated_shares"] = accumulated_shares
    # hold if the current day's price is greater than the previous day's price
    elif row["close"] > 1.02*previous_price:
        AAPL_df.loc[index, "trade_type"] = "sell"
        AAPL_df.loc[index, "cost/proceeds"] = (row["close"] * accumulated_shares)
        accumulated_shares=0
        AAPL_df.loc[index, "accumulated_shares"] = accumulated_shares
        
    # hold if the current day's price is equal to the previous day's price
    else:
        AAPL_df.loc[index, "trade_type"] = "hold"
        AAPL_df.loc[index, "accumulated_shares"] = accumulated_shares
    # update the previous_price to the current row's price
    previous_price = row["close"]

    # if the index is the last index of the DataFrame, sell
    if index == AAPL_df.index[-1]:
        AAPL_df.loc[index, "trade_type"] = "sell"

        # calculate the proceeds by multiplying the last day's price by the accumulated shares
        AAPL_df.loc[index, "cost/proceeds"] = row["close"] * accumulated_shares
        AAPL_df.loc[index, "accumulated_shares"] = accumulated_shares
# Review the DataFrame
AAPL_df


In [ ]:
total_profit_loss = round(AAPL_df["cost/proceeds"].sum(),2)
print(f"The total profit/loss of the trading strategy is ${total_profit_loss}.")

In [ ]:
invested_capital = 0

# Calculate the invested capital by adding the cost of all buy trades
for index, row in AAPL_df.iterrows():
    if row["trade_type"] == "buy":
        invested_capital = invested_capital + row["cost/proceeds"]


# Calculate the return on investment (ROI)
roi = round((total_profit_loss / -(invested_capital)) * 100, 2)

# Print the ROI
print(f"The trading algorithm resulted in a return on investment of {roi}%")

In [ ]:
signals_df = AAPL_df.loc[:,["close"]]

short_window = 50
long_window = 100

signals_df['SMA50'] = signals_df['close'].rolling(window=short_window).mean()
signals_df['SMA100'] = signals_df['close'].rolling(window=long_window).mean()
signals_df['Signal'] = 0.0
signals_df

In [ ]:
signals_df['Signal'][short_window:] = np.where(
    signals_df['SMA50'][short_window:] > signals_df['SMA100'][short_window:], 1.0, 0.0
)
signals_df['Entry/Exit'] = signals_df['Signal'].diff()


In [ ]:
exit = signals_df[signals_df['Entry/Exit'] == -1.0]['close'].hvplot.scatter(
    color='yellow',
    marker='v',
    size=200,
    legend=False,
    ylabel='Price in $',
    width=1000,
    height=400
)
entry = signals_df[signals_df['Entry/Exit'] == 1.0]['close'].hvplot.scatter(
    color='purple',
    marker='^',
    size=200,
    legend=False,
    ylabel='Price in $',
    width=1000,
    height=400
)

# Visualize close price for the investment
security_close = signals_df[['close']].hvplot(
    line_color='lightgray',
    ylabel='Price in $',
    width=1000,
    height=400
)

# Visualize moving averages
moving_avgs = signals_df[['SMA50', 'SMA100']].hvplot(
    ylabel='Price in $',
    width=1000,
    height=400
)

# Create the overlay plot
entry_exit_plot = security_close * moving_avgs * entry * exit

# Show the plot
entry_exit_plot.opts(
    title="Apple - SMA50, SMA100, Entry and Exit Points"
)

In [ ]:
AAPL_bt_df=alpaca.get_bars("AAPL", TimeFrame.Day, "2016-01-02", "2017-04-07", adjustment='raw').df


#AAPL_bt_df=AAPL_bt_df.drop(columns=["open","high","low","volume","trade_count","vwap"])
AAPL_bt_df

In [ ]:
AAPL_df= AAPL_df.rename(columns=str.lower)
AAPL_df

In [ ]:
df = pd.DataFrame()


In [ ]:

# Create a function to find the optimal components for the trading algorithm
# Filter the date index and close columns
def trading_algo(short_window_input, long_window_input, training_period_input):
    signals_df = AAPL_df.loc[:, ["close"]]

    # Use the pct_change function to generate  returns from close prices
    signals_df["Actual Returns"] = signals_df["close"].pct_change()

    # Drop all NaN values from the DataFrame
    signals_df = signals_df.dropna()


    short_window = short_window_input
    long_window =long_window_input

    # Generate the fast and slow simple moving averages (4 and 100 days, respectively)
    signals_df['SMA_Fast'] = signals_df['close'].rolling(window=short_window).mean()
    signals_df['SMA_Slow'] = signals_df['close'].rolling(window=long_window).mean()

    signals_df = signals_df.dropna()


    # Initialize the new Signal column
    signals_df['Signal'] = 0.0

    # When Actual Returns are greater than or equal to 0, generate signal to buy stock long
    signals_df.loc[(signals_df['Actual Returns'] >= 0), 'Signal'] = 1

    # When Actual Returns are less than 0, generate signal to sell stock short
    signals_df.loc[(signals_df['Actual Returns'] < 0), 'Signal'] = 0


    # Calculate the strategy returns and add them to the signals_df DataFrame
    signals_df['Strategy Returns'] = signals_df['Actual Returns'] * signals_df['Signal'].shift()

    # Assign a copy of the sma_fast and sma_slow columns to a features DataFrame called X
    X = signals_df[['SMA_Fast', 'SMA_Slow']].shift().dropna()

    # Create the target set selecting the Signal column and assiging it to y
    y = signals_df['Signal']

    # Select the start of the training period
    training_begin = X.index.min()

    # Select the ending period for the training data with an offset of 3 months
    training_end = X.index.min() + DateOffset(months=training_period_input)


    # Generate the X_train and y_train DataFrames
    X_train = X.loc[training_begin:training_end]
    y_train = y.loc[training_begin:training_end]


    # Generate the X_test and y_test DataFrames
    X_test = X.loc[training_end+DateOffset(hours=1):]
    y_test = y.loc[training_end+DateOffset(hours=1):]


    # Scale the features DataFrames

    # Create a StandardScaler instance
    scaler = StandardScaler()

    # Apply the scaler model to fit the X-train data
    X_scaler = scaler.fit(X_train)

    # Transform the X_train and X_test DataFrames using the X_scaler
    X_train_scaled = X_scaler.transform(X_train)
    X_test_scaled = X_scaler.transform(X_test)
    X_test_scaled

    # From SVM, instantiate SVC classifier model instance
    svm_model = svm.SVC()

    # Fit the model to the data using the training data
    svm_model = svm_model.fit(X_train_scaled, y_train)

    # Use the testing data to make the model predictions
    svm_pred = svm_model.predict(X_test_scaled)


    # Use a classification report to evaluate the model using the predictions and testing data
    svm_testing_report = classification_report(y_test, svm_pred)


    # Create a new empty predictions DataFrame.

    # Create a predictions DataFrame
    predictions_df = pd.DataFrame(index=X_test.index)

    # Add the SVM model predictions to the DataFrame
    predictions_df['Predicted'] = svm_pred

    # Add the actual returns to the DataFrame
    predictions_df['Actual Returns'] = signals_df["Actual Returns"]

    # Add the strategy returns to the DataFrame
    predictions_df['Strategy Returns'] = signals_df["Actual Returns"] * predictions_df["Predicted"]


  
    # Create the column names for the dataframe to find the optimal components for the algorithm. The column names will be generated by the for loop
    df[f'{short_window}_{long_window}_{training_period}'] = (1+predictions_df["Strategy Returns"]).cumprod()
    # Plot the actual returns versus the strategy returns
    #(1+predictions_df[["Actual Returns", "Strategy Returns"]]).cumprod().plot(title= f"{short_window}, {long_window}, {training_period_input}")